# Trajectory Optimization using Direct Collocation

### CS/ECE/ISyE 524 — Introduction to Optimization — Spring 2018  
Alex Dawson-Elli  
Patrick Dills  
Kieran Nichols



### Table of Contents  

* [Introduction](#introduction)  
* [Mathematical modeling](#mathematical modeling)  
* [Exploration of Modeling Assumptions](#assumptions)  
    * [small angles and quazi-static assumptions](#1a)  
    * [small angles assumption](#1b)  
    * [full dynamics](#1c)  
    * [results and discussion](#1RD)
* [Experimentation with Full Dynamics](#2)   
    * [Obsticle avoidance with hard constraints](#2aa)  
    * [Obsticle avoidance with regularization](#2ab)  
    * [Double pendulum swing up](#2b)  
* [Conclusion](#conclusion)  


# Introduction <a class="anchor" id="first-bullet"></a>

Our project involves investigating optimal trajectories for moving a cart with an attached pendulum. This project is usually a typical control theory problem, but we are using optimization to investigate the effects of various assumption like small angle approximations and removal of 2nd order terms in solving the trajectory of the cart while minimizing the motor effort. The cart with mass ($m_1$) lays on a frictionless track and can be moved horizontally by control force ($u$). A pendulum with a massless rod (length, $l$) and a bob (mass, $m_2$) is attached to the cart using a frictionless bearing. The cart and the bob will be treated as point masses. The force acting on the cart will also cause the pendulum to swing. Three models with varying complexity will be used to observe the effects of assumptions on the physical dynamics on three different trajectory scenarios.  
The three models will increase in complexity due to the decrease in the amount of assumptions:
1. Small angle approximation with quazi-static equations. This model will have a quadratic model with linear constraints.
2. Small angle approximation. This model will have a quadratic model with quadratic constraints.
3. No extra assumptions with full dynamic equations. This model will have a quadratic model with non-linear constraints.

The models will aim to minimize the objective function of control force squared when it solves for the control force to move the cart/pendulum through the three trajectory scernarios. All models will have mass, length, state and state derivative (position, velocity, and acceleration) variables for the cart and pendulum. We will use boundary constraints which will limit the initial and final states and state derivatives of the cart and pendulum.

These trajectory scenarios will be swinging the pendulum through the non-inverted positions (rotation of pendulum to a spefic angle?), translation of cart and pendulum while keeping the pendulum in the inverted position, and swing the pendulum from the non-inverted to inverted position (freedom of cart to translate and pendulum to rotate). To verify the solver's ability to estimate the state of the cart and pendulum, a forward euler numerical integrator will also be used. 


# Mathematical modeling <a class="anchor" id="mathematicalModel"></a>

Cartesian coordinates of the cart and pole will be 
\begin{equation}
\begin{bmatrix}
q_1 \\ 0 
\end{bmatrix}
\qquad \text{and} \qquad 
\begin{bmatrix}
q_1 + l \sin(q_2) \\ l \cos(q_2)] 
\end{bmatrix}
\end{equation}
where ($q_1$) is the horizontal position of the cart and ($q_2$) is the angle of the pendulum.

The state, $x$ and state derivative, $\dot{x}$ will give the dynamics of the system in first-order form and represnets the minimal generalized coordinates and its derivatives
\begin{equation}
 x =
 \begin{bmatrix}
  q_1 \\ q_2 \\ \dot{q_1} \\ \dot{q_2}
\end{bmatrix}	
\qquad \text{and} \qquad 		
\dot{x} = f ( x , u ) =
\begin{bmatrix}
\dot{q_1} \\ \dot{q_2} \\ \ddot{q_1} \\ \ddot{q_2}
\end{bmatrix}
\end{equation}

Using Lagrangian method, 	
$ L = T - V $
where T is the kinetic energy and V the potential energy of the system. L summarizes the system dynamics.

\begin{align}
& T = T_{cart} + T_{pole} \\
&T_{cart} = \frac{1}{2} m_1\dot{ q_1}^2 \\
&T_{pole} = \frac{1}{2} m_2  [\dot{(q_1 + l \sin(q_2)}^2 +\dot{(l\cos(q_2)}^2] \\\\
& V = V_{cart} + V_{pole} \\
&V = 0 + m_2 g l \cos(q_2) \\
\end{align}

The potential energy of the cart is zero since it is not moving vertically and it has a height of 0.

Thus $L = \frac{1}{2} m_1 q_1^2 + \frac{1}{2} m_2 [\dot{(x + l \sin(q_2)}^2 +\dot{(l\cos(q_2)}^2] - m_2 g l \cos(q_2)$


The generalized coordinates could be defined as 
\begin{equation}	
X = 
\begin{bmatrix}
q_1  \\ q_2 
\end{bmatrix}
\end{equation}
To yield the equations of motion for the cart/pendulum system, the Lagrangian equations will: 
\begin{align}
&\frac{d}{dt}(\frac{\partial L}{\partial{\dot{q_1}}}) - \frac{\partial L}{\partial{\dot{q_1}}} = u \\
&\frac{d}{dt}(\frac{\partial L}{\partial{\dot{q_2}}}) - \frac{\partial L}{\partial{\dot{q_2}}} = 0
\end{align}

Solving for the partial and time derivatives will yield
\begin{align}
&(m_1 + m_2) \ddot{q_1} + m_1 l \ddot{q_2} \cos(q_2) - m_2 l \dot{q_2}^2 \sin(q_2) = u \\
&m_2 l \ddot{q_1} \cos(q_2) + m_2 l^2 \ddot{q_2}  - m_2 g l \sin(q_2) = 0 
\end{align}

Rearranging the above equations and solving for the $\ddot{q_1}$ and $\ddot{q_2} $ terms respectively,
\begin{align}
&\ddot{q_1} = \frac{ l m_2 \sin(q_2) \dot{q_2}^2 + u + m_{2} g \cos(q_{2}) sin(q_2)} {m_1 + m_2\sin^2(q_2)} \\
&\ddot{q_2} = \frac{ l m_2 \cos(q_2) \sin(q_2^2) \dot{q_2}^2 + u \cos(q_2) + (m_1 + m_2) g \sin(q_2)} {l (m_1 + m_2\sin^2(q_2))}
\end{align}

The objective fuction will be the integral of the actuator effort (control force) squared:		
\begin{equation}
\min_{u(t)} \int_{t_i}^{t_f} u^2 ( \tau ) d \tau
\end{equation}

With discrete constant time steps, the objective fuction will be the sum of the control force from intial to final time.
\begin{equation}
\min_{u(t)} \sum_{t_i}^{t_f} u^2
\end{equation}

Restating this optimization problem in standard form will yield
\begin{align}
\underset{\mathbf{u}_k \in \mathbb{R^2}}{\text{minimize}}\qquad& \sum_{k=1}^{k}\mathbf{u}_k^2\\
\text{subject to:}\qquad& \mathbf{u}_k^2 -U_{max}^2\le 0 && \forall i \in \{1,\dots,k\}\\
%	& \mathbf{q}_{1} - (\mathbf{q}_1)_{init} = 0\\
%	& \mathbf{\dot{q}}_{1} - (\mathbf{\dot q}_1)_{init} = 0\\
%	& \mathbf{q}_{2} - (\mathbf{{q}}_2)_{init} = 0\\
%	& \mathbf{\dot{q}}_{2} - (\mathbf{\dot{q}}_2)_{init} = 0\\
& \mathbf{x} - \mathbf{x}_{init} = 0\\
& \mathbf{\dot{x}} - \mathbf{\dot x}_{init} = 0\\
& \mathbf{x}_{k+1} - \mathbf{x}_{k} - 0.5 h (\mathbf{\dot x}_{k} -\mathbf{\dot x}_{k+1}) = 0 && \forall k \in \{1,\dots,k-1\}\\
%	& \mathbf{\dot{x}}_{k+1} - \mathbf{\dot{x}}_{k} - 
%	= 0 &&\forall k \in \{1,\dots,k-1\}\\
\end{align}
